Extract

In [1]:
import pandas as pd

df = pd.read_excel('Sample - Superstore.xls')
df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


Transform

In [2]:
df.dropna(subset=['Sales', 'Profit', 'Region'], inplace=True)

df['Category'].fillna(df['Category'].mode()[0], inplace=True)


C:\Users\MRITH\AppData\Local\Temp\ipykernel_1744\2414355415.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Category'].fillna(df['Category'].mode()[0], inplace=True)


In [3]:
df.isnull().sum()

Row ID           0
Order ID         0
Order Date       0
Ship Date        0
Ship Mode        0
Customer ID      0
Customer Name    0
Segment          0
Country          0
City             0
State            0
Postal Code      0
Region           0
Product ID       0
Category         0
Sub-Category     0
Product Name     0
Sales            0
Quantity         0
Discount         0
Profit           0
dtype: int64

In [4]:
df.dtypes

Row ID                    int64
Order ID                 object
Order Date       datetime64[ns]
Ship Date        datetime64[ns]
Ship Mode                object
Customer ID              object
Customer Name            object
Segment                  object
Country                  object
City                     object
State                    object
Postal Code               int64
Region                   object
Product ID               object
Category                 object
Sub-Category             object
Product Name             object
Sales                   float64
Quantity                  int64
Discount                float64
Profit                  float64
dtype: object

In [15]:
duplicates = df.duplicated()
print(df[duplicates])


Empty DataFrame
Columns: [Row ID, Order_ID, Order Date, Ship Date, Ship Mode, Customer ID, Customer Name, Segment, Country, City, State, Postal Code, Region, Product ID, Category, Sub-Category, Product Name, Sales, Quantity, Discount, Profit, Shipping Time (Days), Profit Margin (%), Order Month]
Index: []

[0 rows x 24 columns]


In [16]:
df['Sales'] = df['Sales'].astype(float)
df['Profit'] = df['Profit'].astype(float)

In [5]:
df['Order Date'] = pd.to_datetime(df['Order Date'], format='%m/%d/%Y')
df['Ship Date'] = pd.to_datetime(df['Ship Date'], format='%m/%d/%Y')

In [6]:
df['Shipping Time (Days)'] = (df['Ship Date'] - df['Order Date']).dt.days

df['Profit Margin (%)'] = (df['Profit'] / df['Sales']) * 100

In [7]:
df['Order Month'] = df['Order Date'].dt.to_period('M')

monthly_region_sales = df.groupby(['Region', 'Order Month']).agg({
    'Sales': 'sum',
    'Profit': 'sum',
    'Profit Margin (%)': 'mean',
    'Quantity': 'sum',
    'Order ID': 'count' 
}).reset_index()
monthly_region_sales

,Region,Order Month,Sales,Profit,Profit Margin (%),Quantity,Order ID
0,Central,2014-01,1539.906,118.4902,3.275132,62,21
1,Central,2014-02,1233.174,294.8067,-25.763158,69,19
2,Central,2014-03,5827.602,-274.0467,1.334524,123,30
3,Central,2014-04,3712.340,229.3787,20.705782,80,21
4,Central,2014-05,4048.506,-506.0171,-18.134199,121,33
...,...,...,...,...,...,...,...
187,West,2017-08,25737.894,6145.5258,26.777473,338,91
188,West,2017-09,27907.037,4884.5818,22.089524,553,156
189,West,2017-10,21212.436,3283.1081,17.151603,387,96
190,West,2017-11,28941.787,3443.2497,21.594830,556,139


In [8]:
df.rename(columns={'Order ID': 'Order_ID'},inplace=True)

In [9]:
region_sales = df.groupby('Region').agg({
    'Sales': 'sum',
    'Profit': 'sum',
    'Quantity': 'sum',
    'Order_ID': 'count'
}).reset_index()

region_sales['Profit_Margin'] = (region_sales['Profit'] / region_sales['Sales']) * 100

region_sales = region_sales.sort_values(by='Sales', ascending=False)
region_sales

,Region,Sales,Profit,Quantity,Order_ID,Profit_Margin
3,West,725457.8245,108418.4489,12266,3203,14.944831
1,East,678781.2400,91522.7800,10618,2848,13.483399
0,Central,501239.8908,39706.3625,8780,2323,7.921629
2,South,391721.9050,46749.4303,6209,1620,11.934342


In [10]:
region_sales.rename(columns={
    'Order_ID':'Order_Count'
}, inplace=True)

In [11]:
region_sales

,Region,Sales,Profit,Quantity,Order_Count,Profit_Margin
3,West,725457.8245,108418.4489,12266,3203,14.944831
1,East,678781.2400,91522.7800,10618,2848,13.483399
0,Central,501239.8908,39706.3625,8780,2323,7.921629
2,South,391721.9050,46749.4303,6209,1620,11.934342


Load

In [12]:
import sqlite3

conn = sqlite3.connect('Sales_Performance_By_Region.db')
cursor = conn.cursor()

In [13]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS SalesPerformance(
    Region TEXT,
    Sales FLOAT,
    Profit FLOAT,
    Quantity INTEGER,
    Order_Count INTEGER,
    Profit_Margin FLOAT
)
''')

In [14]:
for _, row in region_sales.iterrows():
    cursor.execute('''
    INSERT INTO SalesPerformance (Region, Sales, Profit, Quantity, Order_Count, Profit_Margin)
    VALUES (?, ?, ?, ?, ?, ?)
    ''', (row['Region'], row['Sales'], row['Profit'], row['Quantity'], row['Order_Count'], row['Profit_Margin']))

conn.commit()
conn.close()

### **ETL Summary:**

- **Extract:** Loaded the Superstore dataset from CSV into a pandas DataFrame.

- **Transform:**
  - **Data Cleaning:**
    - Handled missing values by filling them with appropriate defaults (e.g., most common values for categorical columns, zeros for numerical columns).
    - Removed unnecessary columns and duplicates.
    - Converted data types where necessary (e.g., numeric conversion for columns with currency symbols).
  - **Data Transformation:**
    - Calculated additional metrics such as profit margin percentage.
    - Aggregated data by region to compute total sales, total profit, quantity sold, number of orders, and average profit margin.
    - Renamed columns for clarity.

- **Load:**
  - **Database Schema:**
    - **Table Name:** `SalesPerformance`
    - **Columns:**
      - `Region` (TEXT)
      - `Total_Sales` (REAL)
      - `Total_Profit` (REAL)
      - `Quantity_Sold` (INTEGER)
      - `Order_Count` (INTEGER)
      - `Profit_Margin` (REAL)
  - Created the table in an SQLite database.
  - Inserted the transformed data into the SQLite database using Python’s `sqlite3` module.